In [ ]:
import cv2

In [ ]:
import os
os.chdir('/content/drive/My Drive/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.applications.densenet import DenseNet121
from keras.models import Sequential
import keras
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Conv2D, MaxPooling2D, Activation, Flatten

In [ ]:
try:
    x_train = np.load("/content/drive/MyDrive/DR-Data-Blindness_Detection/processed_images/training.npy", mmap_mode=None, allow_pickle=False, fix_imports=True)
    x_validation = np.load("/content/drive/MyDrive/DR-Data-Blindness_Detection/processed_images/validation.npy",mmap_mode=None, allow_pickle=False, fix_imports=True)
    x_test = np.load("/content/drive/MyDrive/DR-Data-Blindness_Detection/processed_images/test.npy", mmap_mode = None,allow_pickle = False, fix_imports = True)
    print("Loaded Successfully...\n")
    print(x_train.shape)
    print(x_validation.shape)
    print(x_test.shape)
except:
    print("file not exists")

Loaded Successfully...

(3112, 512, 512, 3)
(550, 512, 512, 3)
(1928, 512, 512, 3)


In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/DR-Data-Blindness_Detection/training.csv')
train_labels = train_labels['diagnosis']
validation_labels = pd.read_csv('/content/drive/MyDrive/DR-Data-Blindness_Detection/validation.csv')
validation_labels = validation_labels['diagnosis']
print("Training:",train_labels.shape[0])
print("Validation:",validation_labels.shape[0])

Training: 3112
Validation: 550


In [ ]:
### initialization
img_width = 512
img_height = 512
no_channels = 3

In [ ]:
def draw_circle(img):

    '''
    This function is used for drawing a circle from the center of the image.
    Arguments: img(np.array) - image for which we need to draw a circle.
    '''

    x = int(img_width/2)
    y = int(img_height/2)
    r = np.amin((x,y))     # finding radius to draw a circle from the center of the image
    circle_img = np.zeros((img_height, img_width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    return img

In [ ]:
def test_prediction(predicted_labels):
    '''
    Making predictions of the probability scores. The class with more score will be taken as predicted class.
    Arguments:
    predicted_labels - (np.array) - probability score of given sample
    '''
    predicted_labels = predicted_labels > 0.5
    prediction_ordinal = np.empty(predicted_labels.shape, dtype = int)
    prediction_ordinal[:,4] = predicted_labels[:,4]
    for i in range(3, -1, -1): prediction_ordinal[:, i] = np.logical_or(predicted_labels[:,i], prediction_ordinal[:,i+1])
    predicted_labels = prediction_ordinal.sum(axis = 1)-1
    return predicted_labels

In [ ]:
# for model architecture
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Conv2D, MaxPooling2D, Activation, Flatten

In [ ]:
def GAP2D():
    '''Global average pooling layer'''
    global_average_pooling = GlobalAveragePooling2D()
    return global_average_pooling
def dropout(value = 0.5):
    '''Dropout layer'''
    dropout_layer = Dropout(value)
    return dropout_layer
def dense():
    '''Dense layer'''
    dense_layer = Dense(5, activation='sigmoid')
    return dense_layer

In [ ]:
global_average_pooling_layer = GAP2D()
dropout_layer = dropout()
dense_layer = dense()

In [ ]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def DenseNet():
    '''This function is used for building a model architecture of pretrained Densenet121 on imagenet data set.'''
    densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(512,512,3))
    x = global_average_pooling_layer(densenet.layers[-1].output)
    x = dropout_layer(x)
    output = dense_layer(x)
    model = Model(densenet.layers[0].input,output)
    model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(lr=0.00005), metrics=['accuracy'])
    return model

In [ ]:
densenet = DenseNet()
densenet.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_10 (ZeroPadding  (None, 518, 518, 3)  0          ['input_6[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 256, 256, 64  9408        ['zero_padding2d_10[0][0]']      
                                )                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 pool2_bn (BatchNormalization)  (None, 128, 128, 25  1024        ['conv2_block6_concat[0][0]']    
                                6)                                                                
                                                                                                  
 pool2_relu (Activation)        (None, 128, 128, 25  0           ['pool2_bn[0][0]']               
                                6)                                                                
                                                                                                  
 pool2_conv (Conv2D)            (None, 128, 128, 12  32768       ['pool2_relu[0][0]']             
                                8)                                                                
                                                                                                  
 pool2_pool (AveragePooling2D)  (None, 64, 64, 128)  0           ['pool2_conv[0][0]']             
          

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tensorboard = TensorBoard(log_dir = '/content/drive/My Drive/densenet')
kappa_metrics = Metrics('/content/drive/MyDrive/DR-Data-Blindness_Detection/models/densenet.h5')
data_generator = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=180,zoom_range = 0.2)
densenet = DenseNet()
history = densenet.fit_generator(
                   data_generator.flow(x_train, labels_train, batch_size=8),
                    steps_per_epoch=len(x_train) / 8,
                    epochs=30,
                    initial_epoch=0,
                    verbose=1,
                    validation_data=(x_validation, labels_validation),
                    validation_steps=len(x_validation) / 8,
                    callbacks=[kappa_metrics, tensorboard], class_weight = class_weights)